<a href="https://colab.research.google.com/github/phat1505/Health-Classifition/blob/main/Health_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import các thư viện cần thiết
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.offline as pyo
import sklearn
import plotly.express as ex
import warnings
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Đọc dữ liệu từ tệp CSV 'health_classification.csv' và hiển thị 5 dòng đầu tiên của DataFrame
df = pd.read_csv("/content/health_classification.csv")
df.head().T

In [ ]:
# In ra thông tin tổng quan của DataFrame 'df', bao gồm thông tin về cột và kiểu dữ liệu
df.info()

In [ ]:
# Kiểm tra và in ra số lượng giá trị thiếu (NaN) trong từng cột của DataFrame 'df'
df.isna().sum()

In [ ]:
# Hiển thị các thống kê mô tả cơ bản (mean, min, max, etc.) cho các cột có giá trị số
df.describe()

In [ ]:
# Hiển thị hình dạng của DataFrame 'df', tức số dòng và số cột
df.shape

In [7]:
# Xác định các cột phân loại để thực hiện visualizations và thống kê
cols = ['Gender','Blood Type', 'Medical Condition',
        'Insurance Provider', 'Admission Type',
        'Medication', 'Test Results']

In [ ]:
# Đối với các đặc trưng phân loại
for i in cols:

    if i == 'Gender':
        # Vẽ đồ thị cho đặc trưng 'Gender'
        fig, ax = plt.subplots(1, 2)
        fig.suptitle('** Giới tính **', fontsize=20)
        plt.style.use('seaborn')
        plt.subplot(1,2,1)
        df['Gender'].value_counts().plot(kind='bar',color=sns.color_palette("tab10"))
        plt.subplot(1,2,2)
        df['Gender'].value_counts().plot(kind='pie',autopct="%.2f%%")
        plt.show()

    else:
        # Vẽ đồ thị cho các đặc trưng khác
        fig, ax = plt.subplots(1, 2)
        fig.suptitle('** ' + i + ' **', fontsize=20)
        plt.style.use('seaborn')
        plt.subplot(1,2,1)
        df[i].value_counts().plot(kind='bar',color=sns.color_palette("tab10"))
        plt.subplot(1,2,2)
        df[i].value_counts().plot(kind='pie',autopct="%.2f%%")
        plt.show()

In [ ]:
# Tạo DataFrame mới bằng cách nhóm dữ liệu theo cặp cột 'Medical Condition' và 'Medication',
# tính tổng 'Billing Amount' trong mỗi nhóm và reset index
df_trans = df.groupby(['Medical Condition', 'Medication'])[['Billing Amount']].sum().reset_index()

# Tạo biểu đồ cột thể hiện 'Billing Amount' theo 'Medical Condition' và 'Medication',
# sử dụng màu sắc khác nhau cho từng giá trị trong 'Medication'
plt.figure(figsize=(15,6))
sns.barplot(x=df_trans['Medical Condition'], y=df_trans['Billing Amount'], hue=df_trans['Medication'], ci=None, palette="Set1")
# Đặt tiêu đề biểu đồ
plt.title("Billing Amount according to Medical Condition and Medication")
# Đặt tên trục y
plt.ylabel("Billing Amount")
# Xoay nhãn trục x và đặt kích thước font
plt.xticks(rotation=45, fontsize=9)
# Hiển thị biểu đồ
plt.show()

In [ ]:
# Tạo DataFrame mới bằng cách nhóm dữ liệu theo cặp cột 'Medical Condition' và 'Test Results',
# tính tổng 'Billing Amount' trong mỗi nhóm và reset index
df_trans = df.groupby(['Medical Condition', 'Test Results'])[['Billing Amount']].sum().reset_index()

# Tạo biểu đồ cột thể hiện 'Billing Amount' theo 'Medical Condition' và 'Test Results',
# sử dụng màu sắc khác nhau cho từng giá trị trong 'Test Results'
plt.figure(figsize=(15,6))
sns.barplot(x=df_trans['Medical Condition'], y=df_trans['Billing Amount'], hue=df_trans['Test Results'], ci=None, palette="Set1")
# Đặt tiêu đề biểu đồ
plt.title("Billing Amount according to Medical Condition and Test Results")
# Đặt tên trục y
plt.ylabel("Billing Amount")
# Xoay nhãn trục x và đặt kích thước font
plt.xticks(rotation=45, fontsize=9)
# Hiển thị biểu đồ
plt.show()

In [ ]:
# Chuyển đổi 'Date of Admission' và 'Discharge Date' sang kiểu datetime
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'])
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'])

# Tính 'Days hospitalized' là sự chênh lệch thời gian
df['Days hospitalized'] = df['Discharge Date'] - df['Date of Admission']
df.head()

In [ ]:
# Tính trung bình số ngày nhập viện cho mỗi thuộc tính, loại bỏ 'Days hospitalized'
cols = df.columns

for i in cols:
    if i == 'Days hospitalized':
        # Bỏ qua cột 'Days hospitalized'
        continue

    # Tính trung bình số ngày nhập viện cho từng giá trị trong cột 'i'
    char_bar = df.groupby([i])[['Days hospitalized']].mean().reset_index()
    char_bar = char_bar.sort_values(by=("Days hospitalized"), ascending=False)

    top = char_bar.head(10)
    fig = go.Figure(go.Bar(x=top[i], y=top['Days hospitalized']))

    fig.update_layout(title="Highest " + i + " according to average number of days hospitalized",
                      xaxis_title=i,
                      yaxis_title="Days hospitalized",
                      plot_bgcolor='black',
                      paper_bgcolor='gray',
                      font=dict(color='white'))

    # Hiển thị biểu đồ bằng pio trong Colab
    pio.show(fig, renderer="colab")

In [ ]:
# Thay thế giá trị thiếu bằng giá trị trung bình cho các cột số
df.fillna(df.mean(), inplace=True)

# Chọn các cột có liên quan và biến cố mục tiêu
selected_columns = [ 'Gender','Blood Type', 'Medical Condition',
        'Medication', 'Test Results']
data_for_plot = df[selected_columns]

# Chuyển đổi dữ liệu văn bản thành dữ liệu số bằng LabelEncoder
le = LabelEncoder()
for column in data_for_plot.select_dtypes(include=['object']).columns:
    data_for_plot[column] = le.fit_transform(data_for_plot[column])

# Trực quan hóa mỗi thuộc tính so với 'Test Results' bằng countplot
for column in data_for_plot.columns[:-1]:  # Loại bỏ 'Test Results'
    plt.figure(figsize=(25, 8))
    sns.countplot(x=column, hue='Test Results', data=data_for_plot)
    plt.title(f'{column} vs Test Results')
    plt.show()


In [ ]:
#Thuật toán phân cụm

# Thay thế giá trị thiếu bằng giá trị trung bình cho các cột số
df.fillna(df.mean(), inplace=True)

# Chọn các cột số có liên quan để thực hiện phân cụm (loại bỏ 'Test Results')
numeric_columns = df.select_dtypes(include=['number']).columns
numeric_columns = numeric_columns[numeric_columns != 'Test Results']
X = df[numeric_columns]

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Áp dụng thuật toán phân cụm K-Means
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X_scaled)

# Thêm cột 'Cluster' vào DataFrame
df['Cluster'] = kmeans.labels_

# Trực quan hóa kết quả phân cụm
sns.pairplot(df, hue='Cluster', palette='viridis')
plt.show()

In [ ]:
data_for_plot = df[selected_columns]
# Khám phá dữ liệu trong từng cụm cho từng cột với 'Test Results'
for column in selected_columns[:-1]:  # Loại bỏ 'Test Results' vì nó là biến độc lập
    plt.figure(figsize=(10, 6))

    # Thống kê cơ bản cho từng cụm
    for cluster_id in range(n_clusters):
        cluster_data = df[df['Cluster'] == cluster_id]
        cluster_stats = cluster_data[column].describe()

        print(f"Cluster {cluster_id} Statistics for {column}:")
        print(cluster_stats)
        print("\n")

    # Trực quan hóa cụm cho từng cột với 'Test Results'
    sns.scatterplot(data=df, x='Test Results', y=column, hue='Cluster', palette='viridis')

    # Đặt tên cho trục và tiêu đề
    plt.xlabel('Test Results')
    plt.ylabel(column)
    plt.title(f'Clustered Data for {column} vs Test Results')

    # Hiển thị biểu đồ
    plt.show()

In [27]:
#Thuật toán Naive Bayes

# Loại bỏ các hàng có giá trị thiếu
df.dropna(inplace=True)
columns = ['Gender','Blood Type', 'Medical Condition',
        'Medication', 'Test Results']
# Tách dữ liệu thành đặc trưng (X) và biến mục tiêu (y)
X = df[columns]
y = df['Test Results']

# Chuyển đổi dữ liệu văn bản thành dữ liệu số bằng LabelEncoder
le = LabelEncoder()
for column in X.select_dtypes(include=['object']).columns:
    X[column] = le.fit_transform(X[column])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Huấn luyện mô hình Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

In [ ]:
# Thực hiện tinh chỉnh tham số alpha
alphas = [0.1, 0.5, 1.0, 1.5, 2.0]
for alpha in alphas:
    nb_model = MultinomialNB(alpha=alpha)
    nb_model.fit(X_train, y_train)
    y_pred_nb = nb_model.predict(X_test)
    accuracy_nb = accuracy_score(y_test, y_pred_nb)
    print(f'Alpha = {alpha}, Naive Bayes Accuracy: {accuracy_nb}')

# Chọn alpha có độ chính xác cao nhất để huấn luyện mô hình
best_alpha = alphas[np.argmax(accuracy_nb)]

# Huấn luyện mô hình với alpha tốt nhất
nb_model = MultinomialNB(alpha=best_alpha)
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

# In ra độ chính xác và báo cáo phân loại cuối cùng
accuracy_nb_final = accuracy_score(y_test, y_pred_nb)
print(f'Naive Bayes Final Accuracy: {accuracy_nb_final}')
print('Naive Bayes Final Classification Report:')
print(classification_report(y_test, y_pred_nb))